In [ ]:
!pip install rake-nltk

In [ ]:
from rake_nltk import Rake
import nltk

nltk.download('stopwords')
nltk.download('punkt')
nltk.download('punkt_tab')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


True

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.cluster import KMeans
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
from scipy.sparse import hstack, csr_matrix
import ast
from bs4 import BeautifulSoup
import requests
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

rake = Rake()

data = pd.read_csv('IMBD.csv')
columns = ['title', 'description', 'genre', 'stars']
relevant = data[columns].copy()

genre_multiplier = 1
desc_multiplier = 4

genre_dummies = data['genre'].fillna('').str.get_dummies(sep=', ')
genre_sparse = csr_matrix(genre_dummies) * genre_multiplier

stars_clean = data['stars'].str.replace(r"[[\]']", "", regex=True).fillna('')
tfidf_stars = TfidfVectorizer(max_features=500, stop_words='english')
stars_sparse = tfidf_stars.fit_transform(stars_clean)

tfidf_title = TfidfVectorizer(max_features=500, stop_words='english')
title_sparse = tfidf_title.fit_transform(data['title'].fillna(''))

def get_keywords(text):
    if pd.isna(text) or text == "":
        return ""
    rake.extract_keywords_from_text(text)
    return " ".join(rake.get_ranked_phrases()[:5])

relevant['extracted_keywords'] = relevant['description'].apply(get_keywords)

relevant['text_soup'] = (
    relevant['description'].fillna('') + " " +
    (relevant['extracted_keywords'] + " ") * desc_multiplier +
    relevant['genre'].fillna('')
)

tfidf_desc = TfidfVectorizer(max_features=5000, stop_words='english', min_df=2, max_df=0.7)
desc_sparse = tfidf_desc.fit_transform(relevant['text_soup'].fillna(''))

X_combined_sparse = hstack([genre_sparse, stars_sparse, title_sparse, desc_sparse])

svd = TruncatedSVD(n_components=100)
X_reduced = svd.fit_transform(X_combined_sparse)

kmeans = KMeans(n_clusters=20, init='k-means++', n_init=10)
y_kmeans = kmeans.fit_predict(X_reduced)

movie_title = input("Enter Movie Title: ").title()
API_KEY = '40aef168ab16c2f7c59380272ba1b17e'
BASE_URL = 'https://api.themoviedb.org/3'

def get_movie_details(movie_name):
    search_url = f"{BASE_URL}/search/movie"
    params = {
        'api_key': API_KEY,
        'query': movie_name
    }

    response = requests.get(search_url, params=params)
    results = response.json().get('results')

    if not results:
        return "No movie found."

    first_movie = results[0]
    movie_id = first_movie['id']

    detail_url = f"{BASE_URL}/movie/{movie_id}"
    detail_params = {'api_key': API_KEY}
    actual_url = f"https://www.themoviedb.org/movie/{movie_id}"

    movie_details = requests.get(detail_url, params=detail_params).json()
    website = requests.get(actual_url)
    soup = BeautifulSoup(website.text, 'html.parser')

    cast_cards = soup.find_all('li', class_='card')
    actors = [tag.get_text() for tag in cast_cards]
    actors_with_roles = [actor.strip() for actor in actors]
    actors_without_roles = [item.split('\n')[0] for item in actors_with_roles]

    keyword_cards = soup.find_all('section', class_='keywords right_column')
    keywords = [tag.get_text()[11:] for tag in keyword_cards]
    keywords_cleaned = [actor.strip().replace('\n', ', ') for actor in keywords]

    title = movie_details.get('title')
    overview = movie_details.get('overview')
    genre_name = [genre['name'] for genre in movie_details['genres']]

    return {
        'title': title,
        'overview': overview,
        'genre_names': genre_name,
        'actors': actors_without_roles,
        'keywords': keywords_cleaned,
    }

movie_info = get_movie_details(movie_title)

input_title = movie_info['title']
input_overview = movie_info['overview']
input_genre_names = movie_info['genre_names']
input_actors = movie_info['actors']

input_title_sparse = tfidf_title.transform([input_title])
all_keywords = " ".join(movie_info['keywords'])
input_genre_text = " ".join(input_genre_names)
input_text_combined = (movie_info['overview'] or "") + " " + (all_keywords + " ") * desc_multiplier + " " + input_genre_text
input_desc_sparse = tfidf_desc.transform([input_text_combined])

input_genre_str = ', '.join(input_genre_names)
temp_genre_df = pd.DataFrame({'genre_col': [input_genre_str]})
input_genre_encoded = temp_genre_df['genre_col'].str.get_dummies(sep=', ')
input_genre_aligned = input_genre_encoded.reindex(columns=genre_dummies.columns, fill_value=0)
input_genre_sparse = csr_matrix(input_genre_aligned.values) * genre_multiplier

input_actors_str = ' '.join(input_actors)
input_stars_sparse = tfidf_stars.transform([input_actors_str])

X_input_combined = hstack([input_genre_sparse, input_stars_sparse, input_title_sparse, input_desc_sparse])

X_input_reduced = svd.transform(X_input_combined)

y_pred_input = kmeans.predict(X_input_reduced)

relevant['cluster_labels'] = kmeans.labels_
target_cluster = y_pred_input[0]
recommendations = relevant[relevant['cluster_labels'] == target_cluster]
recommend_titles = recommendations['title'].tolist()

print(f"\n--- Recommendation Result ---")
print(f"Movie: {input_title}")
print(f"Assigned Cluster: {y_pred_input[0]}")
print(f"Because you liked '{input_title}', you might also like these movies from Cluster {target_cluster}:")

cluster_indices = relevant[relevant['cluster_labels'] == target_cluster].index
X_cluster_full = X_combined_sparse.tocsr()[cluster_indices]

similarities = cosine_similarity(X_input_combined, X_cluster_full).flatten()

top_indices = similarities.argsort()[::-1]

current_top = []
for idx_in_cluster in top_indices:
    actual_idx = cluster_indices[idx_in_cluster]
    if relevant.iloc[actual_idx]['title'].lower() != input_title.lower():
        current_top.append(idx_in_cluster)
    if len(current_top) == 10:
        break

print(f"\n--- Top 10 Ranked Recommendations (True Precision) ---")
for i, idx_in_cluster in enumerate(current_top):
    actual_idx = cluster_indices[idx_in_cluster]
    row = relevant.iloc[actual_idx]
    score = similarities[idx_in_cluster]
    print(f"{i+1}. {row['title']} | Match: {score:.4f} | Genres: {row['genre']}")


Enter Movie Title: planes

--- Recommendation Result ---
Movie: Planes
Assigned Cluster: 12
Because you liked 'Planes', you might also like these movies from Cluster 12:

--- Top 10 Ranked Recommendations (True Precision) ---
1. Masameer | Match: 0.6221 | Genres: Animation, Adventure, Comedy
2. Dew Drop Diaries | Match: 0.6200 | Genres: Animation, Adventure, Comedy
3. Thelma the Unicorn | Match: 0.6199 | Genres: Animation, Adventure, Comedy
4. Tellur Aliens | Match: 0.6162 | Genres: Animation, Adventure, Comedy
5. Pearl | Match: 0.6140 | Genres: Animation, Adventure, Comedy
6. Not Quite Narwhal | Match: 0.6133 | Genres: Animation, Adventure, Comedy
7. Pashmina | Match: 0.6133 | Genres: Animation, Adventure, Comedy
8. Steps | Match: 0.6131 | Genres: Animation, Adventure, Comedy
9. Bone | Match: 0.6131 | Genres: Animation, Adventure, Comedy
10. Dino Daycare | Match: 0.6130 | Genres: Animation, Adventure, Comedy
